In [2]:
#第3章/加载数据集
from datasets import load_dataset

dataset = load_dataset(path='seamew/ChnSentiCorp')

dataset

Found cached dataset chn_senti_corp (F:/HuggingFaceCache/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [ ]:
#第3章/加载glue数据集
# load_dataset(path='glue', name='sst2', split='train')

In [3]:
#第3章/保存数据集到磁盘
dataset.save_to_disk(
    dataset_dict_path='../data/ChnSentiCorp')

Saving the dataset (0/1 shards):   0%|          | 0/9600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

In [4]:
#第3章/从磁盘加载数据集
from datasets import load_from_disk

dataset = load_from_disk('../data/ChnSentiCorp')

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [5]:
#第3章/使用train数据子集做后续的实验
dataset = dataset['train']

dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 9600
})

In [6]:
#第3章/查看数据样例
for i in [12, 17, 20, 26, 56]:
    print(dataset[i])

{'text': '轻便，方便携带，性能也不错，能满足平时的工作需要，对出差人员来说非常不错', 'label': 1}
{'text': '很好的地理位置，一蹋糊涂的服务，萧条的酒店。', 'label': 0}
{'text': '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', 'label': 1}
{'text': '跟住招待所没什么太大区别。 绝对不会再住第2次的酒店！', 'label': 0}
{'text': '价格太高，性价比不够好。我觉得今后还是去其他酒店比较好。', 'label': 0}


In [7]:
#第3章/排序数据
#数据中的label是无序的
print(dataset['label'][:10])

#让数据按照label排序
sorted_dataset = dataset.sort('label')
print(sorted_dataset['label'][:10])
print(sorted_dataset['label'][-10:])

[1, 1, 0, 0, 1, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
#第3章/打乱数据顺序
shuffled_dataset = sorted_dataset.shuffle(seed=42)

shuffled_dataset['label'][:10]

[0, 1, 0, 0, 1, 0, 1, 0, 1, 0]

In [9]:
#第3章/从数据集中选择某些数据
dataset.select([0, 10, 20, 30, 40, 50])

Dataset({
    features: ['text', 'label'],
    num_rows: 6
})

In [10]:
#第3章/过滤数据
def f(data):
    return data['text'].startswith('非常不错')


dataset.filter(f)

Filter:   0%|          | 0/9600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 13
})

In [11]:
#第3章/切分训练集和测试集
dataset.train_test_split(test_size=0.1)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8640
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 960
    })
})

In [12]:
#第3章/数据分桶
dataset.shard(num_shards=4, index=0)

Dataset({
    features: ['text', 'label'],
    num_rows: 2400
})

In [13]:
#第3章/字段重命名
dataset.rename_column('text', 'text_rename')

Dataset({
    features: ['text_rename', 'label'],
    num_rows: 9600
})

In [14]:
#第3章/删除字段
dataset.remove_columns(['text'])

Dataset({
    features: ['label'],
    num_rows: 9600
})

In [15]:
#第3章/应用函数
def f(data):
    data['text'] = 'My sentence: ' + data['text']
    return data


maped_datatset = dataset.map(f)

print(dataset['text'][20])
print(maped_datatset['text'][20])

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

非常不错，服务很好，位于市中心区，交通方便，不过价格也高！
My sentence: 非常不错，服务很好，位于市中心区，交通方便，不过价格也高！


In [16]:
#第3章/使用批处理加速
def f(data):
    text = data['text']
    text = ['My sentence: ' + i for i in text]
    data['text'] = text
    return data


maped_datatset = dataset.map(function=f,
                             batched=True,
                             batch_size=1000,
                             num_proc=4)

print(dataset['text'][20])
print(maped_datatset['text'][20])

Map (num_proc=4):   0%|          | 0/9600 [00:00<?, ? examples/s]

非常不错，服务很好，位于市中心区，交通方便，不过价格也高！
My sentence: 非常不错，服务很好，位于市中心区，交通方便，不过价格也高！


In [17]:
#第3章/设置数据格式
dataset.set_format(type='torch', columns=['label'], output_all_columns=True)

dataset[20]

{'label': tensor(1), 'text': '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！'}

In [18]:
#第3章/导出为csv格式
dataset = load_dataset(path='seamew/ChnSentiCorp', split='train')
dataset.to_csv(path_or_buf='../data/ChnSentiCorp.csv')

#加载csv格式数据
csv_dataset = load_dataset(path='csv',
                           data_files='../data/ChnSentiCorp.csv',
                           split='train')
csv_dataset[20]

Found cached dataset chn_senti_corp (F:/HuggingFaceCache/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to F:/HuggingFaceCache/csv/default-874353b92dd6f43c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


{'text': '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', 'label': 1}

In [19]:
#第3章/导出为json格式
dataset = load_dataset(path='seamew/ChnSentiCorp', split='train')
dataset.to_json(path_or_buf='../data/ChnSentiCorp.json')

#加载json格式数据
json_dataset = load_dataset(path='json',
                            data_files='../data/ChnSentiCorp.json',
                            split='train')
json_dataset[20]

Found cached dataset chn_senti_corp (F:/HuggingFaceCache/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to F:/HuggingFaceCache/json/default-1065965fa3acec18/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


{'text': '非常不错，服务很好，位于市中心区，交通方便，不过价格也高！', 'label': 1}